## Load ERA5 data


In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import requests
import h5py
import sys; sys.path.append('../')
import openai

from backend.geoloc import geocode, get_closest_pixel

In [2]:
data_dir = Path('../data/era5/')

# Open h5py file
f = h5py.File(data_dir / 'sample.h5', 'r')

In [3]:
# Load u wind, v wind, and temperature
u = f['u10'][0]
v = f['v10'][0]
temp = f['t2m'][0]

In [5]:
# Image represents the Earth: get closest pixel to target location
data = geocode(city='San Francisco', state='CA', country_code='US')
lat_idx, lon_idx = get_closest_pixel(data['lat'], data['lon'])

# Get the data
u_sf = u[lat_idx, lon_idx]
v_sf = v[lat_idx, lon_idx]
temp_sf = temp[lat_idx, lon_idx]
print('Wind speed: {:.2f} [m/s]'.format(np.sqrt(u_sf**2 + v_sf**2)))
print("Temperature: {:.2f} [C]".format(temp_sf - 273.15)) # kelvin to celsius


Wind speed: 4.40 [m/s]
Temperature: 15.63 [C]


In [34]:
def crop_data(x, lat_idx, lon_idx, num_pixels=100, max_lon_idx=1440, max_lat_idx=721, return_idxs=False):
    """
    Given a 2D array, crop the data around a given location with a given number of pixels.
    We implement checks to make sure we don't go out of bounds.
    """
    lat_min = lat_idx - int(num_pixels / 2)
    lat_max = lat_idx + int(num_pixels / 2)
    lon_min = lon_idx - num_pixels
    lon_max = lon_idx + num_pixels

    if lat_min < 0:
        lat_min = 0
    if lat_max > max_lat_idx:
        lat_max = max_lat_idx
    if lon_min < 0:
        lon_min += max_lon_idx
    if lon_max > max_lon_idx:
        lon_max -= max_lon_idx

    # Get indexes of the data
    if lat_min < lat_max:
        lat_idxs = np.arange(lat_min, lat_max)
    else:
        lat_idxs = np.concatenate((np.arange(lat_min, max_lat_idx), np.arange(0, lat_max)))

    if lon_min < lon_max:
        lon_idxs = np.arange(lon_min, lon_max)
    else:
        lon_idxs = np.concatenate((np.arange(lon_min, max_lon_idx), np.arange(0, lon_max)))
    if return_idxs:
        return x[:, lon_idxs][lat_idxs, :], lat_idxs, lon_idxs
    else:
        return x[:, lon_idxs][lat_idxs, :]


# dat = np.sqrt(f['v10'][0]**2 + f['u10'][0]**2) 
dat = f['tp'][0]
temp_sf, lat_idxs, lon_idxs = crop_data(dat, lat_idx, lon_idx, num_pixels=100, return_idxs=True)


print(temp_sf.shape)
print(lat_idxs.shape)
print(lon_idxs.shape)


(100, 200)
(100,)
(200,)


In [35]:
# Get latitudes and longitudes with the same shape as the data
import pandas as pd


lats = np.linspace(90, -90, 721)
longs = np.linspace(-180, 180, 1440)

# Get coordinates of the data we cropped
lats_sf = lats[lat_idxs]
longs_sf = longs[lon_idxs]

# Make meshgrid
lats_sf, longs_sf = np.meshgrid(lats_sf, longs_sf)

# Create pandas dataframe with columns temp_sf, longitudes, and latitudes
print(temp_sf.shape, lats_sf.shape, longs_sf.shape)

df = pd.DataFrame({'z': temp_sf.flatten(), 'longitude': longs_sf.flatten(), 'latitude': lats_sf.flatten()})


(100, 200) (200, 100) (200, 100)


### Save for `pydeck`

In [36]:
#  save dataframe to csv
df.to_csv('temp_sf.csv', index=False)